In [1]:
LANDING = "https://en.wiktionary.org/wiki/Category:Hungarian_terms_with_audio_links"

In [3]:
!pip install bs4

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)


In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
rq = requests.get(LANDING)

In [6]:
assert rq.status_code == 200

In [7]:
rq.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Category:Hungarian terms with audio links - Wiktionary, the free dictionary</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"886875b4-c049-4966-95b2-12110eac84e5","wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Category:Hungarian_terms_with_audio_links","wgTitle":"Hungarian terms with audio links","wgCurRevisionId":47076145,"wgRevisionId":47076145,"wgArticleId":4376193,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages using DynamicPageList","Hungarian entry maintenance","Terms

In [8]:
soup = BeautifulSoup(rq.text)

In [18]:
cat = soup.find("div", {"class": "mw-category-generated"})

In [ ]:
cat.findAll('li')

In [26]:
def get_category(soup: BeautifulSoup):
    links = []
    cat = soup.find("div", {"class": "mw-category-generated"})
    for link in cat.findAll('li'):
        links.append(f"https://en.wiktionary.org{link.find('a')['href']}")
    return links

In [33]:
def get_next_link(soup: BeautifulSoup):
    cat = soup.find("div", {"class": "mw-category-generated"})
    for a in cat.findAll('a'):
        if a.text.strip() == 'next page':
            return f"https://en.wiktionary.org{a['href']}"
    return None

In [52]:
import time
links = []
next_page = LANDING
while next_page is not None:
    req = requests.get(next_page)
    soup = BeautifulSoup(req.text)
    links += get_category(soup)
    next_page = get_next_link(soup)
    print(next_page)
    del soup
    time.sleep(3)
    

https://en.wiktionary.org/w/index.php?title=Category:Hungarian_terms_with_audio_links&pagefrom=ALKIRA%EF%80%80L%EF%80%80SA%EF%80%80G%0Aalkir%C3%A1lys%C3%A1g#mw-pages
https://en.wiktionary.org/w/index.php?title=Category:Hungarian_terms_with_audio_links&pagefrom=A%EF%80%80LLAM%0A%C3%A1llam#mw-pages
https://en.wiktionary.org/w/index.php?title=Category:Hungarian_terms_with_audio_links&pagefrom=BELJEBB%0Abeljebb#mw-pages
https://en.wiktionary.org/w/index.php?title=Category:Hungarian_terms_with_audio_links&pagefrom=BUTA%0Abuta#mw-pages
https://en.wiktionary.org/w/index.php?title=Category:Hungarian_terms_with_audio_links&pagefrom=DADAIZMUS%0Adadaizmus#mw-pages
https://en.wiktionary.org/w/index.php?title=Category:Hungarian_terms_with_audio_links&pagefrom=DU%EF%80%82NE%0Ad%C5%B1ne#mw-pages


In [47]:
type(soup)

NameError: name 'soup' is not defined

In [48]:
get_next_link(soup)

NameError: name 'soup' is not defined

In [50]:
req = requests.get(next_page)

In [51]:
soup = BeautifulSoup(req.text)

In [40]:
next_page = get_next_link(soup)

In [43]:
# https://en.wiktionary.org/w/index.php?title=Category:Hungarian_terms_with_audio_links&pagefrom=ALKIRA%EF%80%80L%EF%80%80SA%EF%80%80G%0Aalkir%C3%A1lys%C3%A1g#mw-pages
next_page

'https://en.wiktionary.org/w/index.php?title=Category:Hungarian_terms_with_audio_links&pagefrom=ALKIRA%EF%80%80L%EF%80%80SA%EF%80%80G%0Aalkir%C3%A1lys%C3%A1g#mw-pages'